# Inerpretation-Net Training

# Experiment 1: I-Net Performance for Different Algebras and Complexities
# Experiment 2: I-Net Performance Comparison for λ-Nets with Different Training Levels
# Experiment 3: I-Net Performance Comparison Different Training Data Sizes

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'data': {
        'd': 5, #degree
        'n': 1, #number of variables
        'sparsity': None,
        'sample_sparsity': None,
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform', 'beta', 'Gamma', 'laplace'
        'a_max': 1,
        'a_min': -1,
        'lambda_nets_total': 10000,
        'noise': 0,
        'noise_distrib': 'normal', #'normal', 'uniform', 'beta', 'Gamma', 'laplace'
        
        'same_training_all_lambda_nets': False,

        'fixed_seed_lambda_training': True,
        'fixed_initialization_lambda_training': False,
        'number_different_lambda_trainings': 1,
    },
    'lambda_net': {
        'epochs_lambda': 1000,
        'early_stopping_lambda': True,  #if early stopping is used, multi_epoch_analysis is deactivated
        'batch_lambda': 64,
        'dropout': 0,
        'lambda_network_layers': [5*'sparsity'],
        'optimizer_lambda': 'adam',
        'loss_lambda': 'mae',
        'number_of_lambda_weights': None,
        'lambda_dataset_size': 1000,
    },
    'i_net': {
        'optimizer': 'custom',#adam
        'inet_loss': 'mae',
        'inet_metrics': ['r2'],
        'dropout': 0.25,
        'epochs': 500,
        'early_stopping': True,
        'batch_size': 512,
        'dense_layers': [512, 1024],
        'convolution_layers': None,
        'lstm_layers': None,
        'interpretation_dataset_size': 10000,
                
        'interpretation_net_output_monomials': 3, #(None, int)
        'interpretation_net_output_shape': None, #calculated automatically later
        
        'evaluate_with_real_function': False,
        'consider_labels_training': False,
                      
        'data_reshape_version': None, #default to 2 options:(None, 0,1 2)
        'nas': True,
        'nas_type': 'CNN-LSTM-parallel', #options:(None, 'CNN', 'LSTM', 'CNN-LSTM', 'CNN-LSTM-parallel')      
        'nas_trials': 100,
    },
    'evaluation': {   
        'inet_holdout_seed_evaluation': False,
        
        #set if multi_epoch_analysis should be performed
        'multi_epoch_analysis': True,
        'each_epochs_save_lambda': 20,
        'epoch_start': 0, #use to skip first epochs in multi_epoch_analysis
        
        #set if samples analysis should be performed
        'samples_list': None,#[100, 500, 750, 1000, 2500, 5000, 7500, 10000, 15000, 20000, 25000, 28125] 
       
        'random_evaluation_dataset_size': 500,
    },
    'computation':{
        'n_jobs': 1,
        'use_gpu': True,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}

## Imports

In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

In [3]:
#######################################################################################################################################
##################################################### IMPORT LIBRARIES ################################################################
#######################################################################################################################################
from itertools import product       
from tqdm import tqdm_notebook as tqdm
import pickle
import numpy as np
import pandas as pd
import scipy as sp
import timeit
import psutil

from functools import reduce
from more_itertools import random_product 
from sklearn.preprocessing import Normalizer


import os
import shutil

import logging

from prettytable import PrettyTable
import colored
import math

import time
from datetime import datetime
from collections.abc import Iterable


from joblib import Parallel, delayed

from scipy.integrate import quad

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_absolute_error, r2_score
from similaritymeasures import frechet_dist, area_between_two_curves, dtw
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.utils import plot_model
from IPython.display import Image

import keras.backend as K
from livelossplot import PlotLossesKerasTF
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from matplotlib import pyplot as plt
import seaborn as sns


import tensorflow as tf
import random 


import warnings


from IPython.display import display, Math, Latex



In [ ]:
sys.path.append('..')

from utilities.InterpretationNet import *
from utilities.LambdaNet import *
from utilities.metrics import *
from utilities.utility_functions import *

In [4]:
#######################################################################################################################################
################################################### VARIABLE ADJUSTMENTS ##############################################################
#######################################################################################################################################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]

n_jobs = min((epochs_lambda//each_epochs_save_lambda+1, n_jobs)) if multi_epoch_analysis else min(len(samples_list), n_jobs) if samples_list!=None else 1

multi_epoch_analysis = False if early_stopping_lambda else multi_epoch_analysis #deactivate multi_epoch_analysis if early stopping is used

each_epochs_save_lambda = each_epochs_save_lambda if multi_epoch_analysis else epochs_lambda
epochs_save_range_lambda = range(epoch_start//each_epochs_save_lambda, epochs_lambda//each_epochs_save_lambda) if each_epochs_save_lambda == 1 else range(epoch_start//each_epochs_save_lambda, epochs_lambda//each_epochs_save_lambda+1) if multi_epoch_analysis else range(1,2)

data_reshape_version = 2 if data_reshape_version == None and (convolution_layers != None or lstm_layers != None or (nas and nas_type != 'SEQUENTIAL')) else data_reshape_version
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers if use_gpu else ''

logging.getLogger('tensorflow').disabled = True

sns.set_style("darkgrid")
#np.set_printoptions(suppress=True)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if int(tf.__version__[0]) >= 2:
    tf.random.set_seed(RANDOM_SEED)
else:
    tf.set_random_seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [5]:
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################
config['evaluation']['multi_epoch_analysis'] = multi_epoch_analysis

config['evaluation']['each_epochs_save_lambda'] = each_epochs_save_lambda

config['i_net']['data_reshape_version'] = data_reshape_version

config['data']['sparsity'] = nCr(config['data']['n']+config['data']['d'], config['data']['d'])
config['data']['sample_sparsity'] = config['data']['sparsity'] if config['data']['sample_sparsity'] == None else config['data']['sample_sparsity']

config['i_net']['interpretation_net_output_shape'] = config['data']['sparsity'] if config['i_net']['interpretation_net_output_monomials'] is None else 2*config['i_net']['interpretation_net_output_monomials']


transformed_layers = []
for layer in config['lambda_net']['lambda_network_layers']:
    if type(layer) == str:
        transformed_layers.append(layer.count('sparsity')*config['data']['sparsity'])
    else:
        transformed_layers.append(layer)
config['lambda_net']['lambda_network_layers'] = transformed_layers

layers_with_input_output = list(flatten([[config['data']['n']], config['lambda_net']['lambda_network_layers'], [1]]))
number_of_lambda_weights = 0
for i in range(len(layers_with_input_output)-1):
    number_of_lambda_weights += (layers_with_input_output[i]+1)*layers_with_input_output[i+1]  
config['lambda_net']['number_of_lambda_weights'] = number_of_lambda_weights
    
#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])



initialize_LambdaNet_config_from_curent_notebook(config)
initialize_metrics_config_from_curent_notebook(config)
initialize_utility_functions_config_from_curent_notebook(config)
initialize_InterpretationNet_config_from_curent_notebook(config)


#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(path_type='interpretation_net'))
create_folders_inet()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [6]:
print(path_identifier_interpretation_net_data)

print(path_identifier_lambda_net_data)


inet_dense[512, 1024]convNonelstmNoneoutput_6_drop0.25e500b512_customlnets_100_30-1000eES64b_adam_mae_train_1000_var_1_d_5_sparsity_6_amin_-1_amax_1_xmin_0_xmax_1_xdist_uniform_noise_normal_0_diffX_1-FixSeed_42
lnets_100_30-1000eES64b_adam_mae_train_1000_var_1_d_5_sparsity_6_amin_-1_amax_1_xmin_0_xmax_1_xdist_uniform_noise_normal_0_diffX_1-FixSeed_42


In [7]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  1
Num XLA-GPUs Available:  1


## Utility functions

### Generate List of Monomial Identifiers

In [8]:
list_of_monomial_identifiers_extended = []
for i in tqdm(range((d+1)**n)):    
    monomial_identifier = dec_to_base(i, base = (d+1)).zfill(n) 
    list_of_monomial_identifiers_extended.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    monomial_identifier_values = list(map(int, list(monomial_identifier)))
    if sum(monomial_identifier_values) <= d:
        list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)


layers_with_input_output = list(flatten([[n], lambda_network_layers, [1]]))
number_of_lambda_weights = 0
for i in range(len(layers_with_input_output)-1):
    number_of_lambda_weights += (layers_with_input_output[i]+1)*layers_with_input_output[i+1]

  0%|          | 0/6 [00:00<?, ?it/s]

List length: 6
Number of monomials in a polynomial with 1 variables and degree 5: 6
Sparsity: 6
['0', '1', '2', '3', '4', '5']


  0%|          | 0/6 [00:00<?, ?it/s]

List length: 6
Number of monomials in a polynomial with 1 variables and degree 5: 6
Sparsity: 6
['0', '1', '2', '3', '4', '5']


## Load Data and Generate Datasets

In [9]:
def load_lambda_nets(index):
    
    if psutil.virtual_memory().percent > 80:
        raise SystemExit("Out of RAM!")
    
    path = './data/weights/' + 'weights_' + path_identifier_lambda_net_data + '/weights_' + '_epoch_' + str(index).zfill(3) + '.txt'

    
    weight_data = pd.read_csv(path, sep=",", header=None)
    weight_data = weight_data.sort_values(by=0).sample(frac=1, random_state=RANDOM_SEED)
    weight_data = weight_data.sort_values(by=0).sample(n=interpretation_dataset_size, random_state=RANDOM_SEED)

    lambda_nets = []
    for _, row in weight_data.iterrows():
        lambda_net = LambdaNet(row.values)
        lambda_nets.append(lambda_net)
        
    lambda_net_dataset = LambdaNetDataset(lambda_nets)
        
    return lambda_net_dataset
    

In [10]:
#LOAD DATA

parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
lambda_net_dataset_list = parallel(delayed(load_lambda_nets)((i+1)*each_epochs_save_lambda if each_epochs_save_lambda==1 else i*each_epochs_save_lambda if i > 1 else each_epochs_save_lambda if i==1 else 1) for i in epochs_save_range_lambda)  
del parallel

lambda_net_dataset = lambda_net_dataset_list[-1]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


## Data Inspection

In [11]:
lambda_net_dataset.as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
83,1373158606,-0.684,0.543,0.184,0.902,-0.758,0.033,-0.683,0.532,0.047,1.829,-2.399,0.918,-0.683,0.532,0.047,1.829,-2.399,0.918,-0.009,-0.227,0.139,0.026,0.385,0.348,0.091,-0.297,-0.009,0.416,0.274,-0.088,-0.019,0.144,0.349,0.254,0.207,0.416,0.484,0.444,0.273,-0.065,-0.155,-0.042,-0.404,-0.090,-0.051,-0.291,-0.209,-0.233,0.000,0.000,0.154,0.164,-0.091,-0.001,0.157,0.000,0.167,-0.001,0.139,0.000,0.000,0.150,-0.001,0.147,-0.001,-0.193,-0.042,-0.168,-0.001,0.171,0.213,0.000,0.000,0.000,0.169,0.000,0.000,0.000,-0.254,-0.359,-0.297,-0.378,0.381,0.427,-0.165,-0.408,-0.435,0.102,-0.107,-0.352,-0.364,-0.150,0.363,-0.191,0.634,0.281,0.417,0.319,0.287,-0.530,-0.314,-0.261,-0.383,-0.059,-0.544,0.144,-0.258,-0.315,-0.157
53,1373158606,-0.748,-0.121,-0.675,0.502,-0.667,-0.035,-0.586,-0.949,0.445,-0.963,0.940,-0.339,-0.586,-0.949,0.445,-0.963,0.940,-0.339,-0.009,-0.227,0.353,0.246,0.191,0.080,0.309,-0.297,0.214,0.378,0.477,-0.088,-0.019,0.356,0.078,0.462,0.008,0.236,0.276,0.254,0.033,0.161,0.102,-0.042,-0.404,-0.090,0.173,-0.291,-0.209,-0.233,0.000,0.000,0.128,0.129,-0.009,-0.081,0.136,0.000,0.129,0.156,0.133,0.000,0.000,0.134,-0.080,0.130,-0.023,-0.022,-0.017,-0.018,-0.040,0.128,0.149,0.000,0.000,0.000,0.128,0.000,0.000,0.000,-0.254,-0.359,-0.389,-0.390,0.157,0.187,-0.232,-0.408,-0.417,-0.120,-0.251,-0.352,-0.364,-0.248,0.121,-0.328,0.393,0.041,0.204,0.084,0.049,-0.463,-0.199,-0.261,-0.383,-0.059,-0.489,0.144,-0.258,-0.315,-0.120
70,1373158606,-0.134,0.691,-0.829,-0.233,-0.994,0.455,-0.102,0.262,1.005,-3.562,1.648,-0.285,-0.102,0.262,1.005,-3.562,1.648,-0.285,-0.009,-0.227,0.462,0.390,0.251,0.186,0.442,-0.297,0.369,0.542,0.575,-0.088,-0.019,0.468,0.188,0.539,-0.004,0.318,0.334,0.325,0.112,0.338,0.371,-0.042,-0.404,-0.090,0.342,-0.291,-0.209,-0.233,0.000,0.000,-0.189,-0.251,-0.003,-0.002,-0.261,0.000,-0.262,-0.292,-0.148,0.000,0.000,-0.225,-0.002,-0.175,-0.005,-0.003,-0.004,-0.004,-0.001,-0.281,-0.329,0.000,0.000,0.000,-0.263,0.000,0.000,0.000,-0.254,-0.359,-0.632,-0.796,0.231,0.263,-0.580,-0.408,-0.875,-0.449,-0.412,-0.352,-0.364,-0.520,0.197,-0.489,0.358,0.111,0.268,0.151,0.103,-1.013,-0.885,-0.261,-0.383,-0.059,-1.004,0.144,-0.258,-0.315,-0.099
45,1373158606,-0.129,-0.355,-0.886,0.057,-0.600,0.880,-0.108,-0.718,1.330,-6.280,7.722,-3.074,-0.108,-0.718,1.330,-6.280,7.722,-3.074,-0.009,-0.227,0.335,0.229,0.164,0.078,0.295,-0.297,0.196,0.660,0.461,-0.088,-0.019,0.341,0.077,0.444,0.008,0.364,0.276,0.250,0.033,0.143,0.421,-0.042,-0.404,-0.090,0.155,-0.291,-0.209,-0.233,0.000,0.000,0.028,0.028,-0.117,-0.080,0.024,0.000,0.029,-0.249,0.026,0.000,0.000,0.025,-0.078,0.028,-0.023,-0.079,-0.001,-0.001,-0.039,0.029,-0.112,0.000,0.000,0.000,0.029,0.000,0.000,0.000,-0.254,-0.359,-0.341,-0.333,0.206,0.184,-0.179,-0.408,-0.356,-0.362,-0.208,-0.352,-0.364,-0.198,0.118,-0.285,0.393,-0.192,0.203,0.076,0.048,-0.392,-0.547,-0.261,-0.383,-0.059,-0.421,0.144,-0.258,-0.315,-0.032
44,1373158606,0.399,0.079,-0.881,-0.161,-0.337,0.605,0.398,0.055,-0.391,-2.230,2.795,-0.956,0.398,0.055,-0.391,-2.230,2.795,-0.956,-0.009,-0.227,0.385,0.275,0.141,0.076,0.342,-0.297,0.236,0.425,0.526,-0.088,-0.019,0.390,0.071,0.507,-0.071,0.193,0.230,0.210,-0.042,0.183,0.170,-0.042,-0.404,-0.090,0.197,-0.291,-0.209,-0.233,0.00

In [12]:
lambda_net_dataset.as_pandas().describe()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
count,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000
mean,1373158606.000,0.012,-0.058,-0.027,0.140,0.032,-0.014,0.003,-0.029,0.226,-1.100,1.855,-0.890,0.003,-0.029,0.226,-1.100,1.855,-0.890,-0.009,-0.227,0.298,0.205,0.295,0.242,0.285,-0.297,0.168,0.494,0.497,-0.088,-0.019,0.329,0.247,0.405,0.098,0.418,0.375,0.402,0.187,0.119,0.074,-0.042,-0.404,-0.090,0.133,-0.291,-0.209,-0.233,0.000,0.000,-0.018,-0.022,-0.013,-0.027,-0.055,0.000,-0.018,-0.120,-0.054,0.000,0.000,-0.052,-0.036,-0.009,-0.003,-0.069,0.021,-0.021,-0.041,-0.022,-0.018,0.000,0.000,0.000,-0.022,0.000,0.000,0.000,-0.254,-0.359,-0.386,-0.422,0.358,0.426,-0.247,-0.408,-0.447,0.077,-0.102,-0.352,-0.364,-0.224,0.374,-0.290,0.632,0.142,0.350,0.193,0.351,-0.505,-0.261,-0.261,-0.383,-0.059,-0.533,0.144,-0.258,-0.315,0.001
std,0.000,0.561,0.599,0.586,0.606,0.618,0.565,0.527,0.596,2.039,7.059,10.241,4.544,0.527,0.596,2.039,7.059,10.241,4.544,0.000,0.000,0.140,0.142,0.160,0.161,0.164,0.000,0.141,0.321,0.256,0.000,0.000,0.171,0.175,0.137,0.159,0.205,0.162,0.177,0.225,0.139,0.210,0.000,0.000,0.000,0.138,0.000,0.000,0.000,0.000,0.000,0.151,0.154,0.169,0.184,0.187,0.000,0.150,0.333,0.243,0.000,0.000,0.195,0.201,0.140,0.176,0.246,0.125,0.184,0.242,0.156,0.199,0.000,0.000,0.000,0.151,0.000,0.000,0.000,0.000,0.000,0.188,0.235,0.223,0.271,0.274,0.000,0.237,0.491,0.476,0.000,0.000,0.318,0.289,0.159,0.351,0.417,0.175,0.337,0.389,0.278,0.350,0.000,0.000,0.000,0.271,0.000,0.000,0.000,0.143
min,1373158606.000,-0.976,-0.977,-0.974,-0.999,-0.999,-0.980,-0.974,-1.716,-4.105,-36.540,-17.623,-23.340,-0.974,-1.716,-4.105,-36.540,-17.623,-23.340,-0.009,-0.227,-0.031,-0.181,-0.035,-0.113,-0.025,-0.297,-0.227,0.011,0.209,-0.088,-0.019,0.043,-0.124,0.128,-0.197,0.057,0.071,0.061,-0.217,-0.147,-0.276,-0.042,-0.404,-0.090,-0.131,-0.291,-0.209,-0.233,0.000,0.000,-0.476,-0.389,-0.452,-0.512,-0.527,0.000,-0.409,-0.999,-1.045,0.000,0.000,-0.700,-0.568,-0.413,-0.501,-0.845,-0.264,-0.689,-0.663,-0.442,-0.561,0.000,0.000,0.000,-0.415,0.000,0.000,0.000,-0.254,-0.359,-1.196,-1.172,0.099,0.139,-0.887,-0.408,-1.254,-0.951,-0.833,-0.352,-0.364,-1.137,0.072,-0.899,0.358,-1.530,0.028,-1.351,0.033,-1.412,-1.483,-0.261,-0.383,-0.059,-1.397,0.144,-0.258,-0.315,-0.372
25%,1373158606.000,-0.431,-0.580,-0.574,-0.328,-0.549,-0.489,-0.376,-0.390,-0.810,-2.487,-2.403,-1.172,-0.376,-0.390,-0.810,-2.487,-2.403,-1.172,-0.009,-0.227,0.205,0.086,0.181,

## Generate Datasets for Interpretation-Net training

In [13]:
#generate train, test and validation data for training

lambda_net_train_dataset_list = []
lambda_net_valid_dataset_list = []
lambda_net_test_dataset_list = []

for lambda_net_dataset in lambda_net_dataset_list:
    
    
    if inet_holdout_seed_evaluation:
        complete_seed_list = list(set(lambda_net_dataset.train_settings_list['seed']))#list(weight_data.iloc[:,1].unique())

        random.seed(RANDOM_SEED)
        test_seeds = random.sample(complete_seed_list, int(len(complete_seed_list)-len(complete_seed_list)/(1/0.9)))
        lambda_net_test_dataset = lambda_net_dataset.get_lambda_nets_by_seed(test_seeds)
        complete_seed_list = list(set(complete_seed_list) - set(test_seeds))#complete_seed_list.remove(test_seeds)
        
        random.seed(RANDOM_SEED)
        valid_seeds = random.sample(complete_seed_list, int(len(complete_seed_list)-len(complete_seed_list)/(1/0.9)))
        lambda_net_valid_dataset = lambda_net_dataset.get_lambda_nets_by_seed(valid_seeds)
        complete_seed_list = list(set(complete_seed_list) - set(valid_seeds))

        train_seeds = complete_seed_list
        lambda_net_train_dataset = lambda_net_dataset.get_lambda_nets_by_seed(train_seeds)       
        
        lambda_net_train_dataset_list.append(lambda_net_train_dataset)
        lambda_net_valid_dataset_list.append(lambda_net_valid_dataset)
        lambda_net_test_dataset_list.append(lambda_net_test_dataset)
        
        del lambda_net_dataset
    else:
        lambda_net_train_with_valid_dataset, lambda_net_test_dataset = split_LambdaNetDataset(lambda_net_dataset, test_split=0.1)
        lambda_net_train_dataset, lambda_net_valid_dataset = split_LambdaNetDataset(lambda_net_train_with_valid_dataset, test_split=0.1)

        lambda_net_train_dataset_list.append(lambda_net_train_dataset)
        lambda_net_valid_dataset_list.append(lambda_net_valid_dataset)
        lambda_net_test_dataset_list.append(lambda_net_test_dataset)
    
        del lambda_net_dataset, lambda_net_train_with_valid_dataset

        
del lambda_net_dataset_list

In [14]:
lambda_net_train_dataset_list[-1].as_pandas().shape

(81, 110)

In [15]:
lambda_net_valid_dataset_list[-1].as_pandas().shape

(9, 110)

In [16]:
lambda_net_test_dataset_list[-1].as_pandas().shape

(10, 110)

In [17]:
lambda_net_train_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
10,1373158606,-0.816,0.074,-0.689,0.409,0.026,0.996,-0.795,-0.390,2.245,-7.351,9.045,-2.801,-0.795,-0.390,2.245,-7.351,9.045,-2.801,-0.009,-0.227,0.168,0.050,0.501,0.474,0.104,-0.297,0.014,0.131,0.297,-0.088,-0.019,0.162,0.501,0.283,0.011,0.606,0.522,0.569,0.565,-0.045,-0.198,-0.042,-0.404,-0.090,-0.028,-0.291,-0.209,-0.233,0.000,0.000,0.173,0.178,-0.343,-0.375,0.186,0.000,0.179,0.222,0.175,0.000,0.000,0.181,-0.420,0.172,-0.021,-0.452,-0.264,-0.349,-0.492,0.182,0.246,0.000,0.000,0.000,0.179,0.000,0.000,0.000,-0.254,-0.359,-0.325,-0.379,0.874,1.051,-0.194,-0.408,-0.429,-0.081,-0.157,-0.352,-0.364,-0.188,1.025,-0.235,0.395,0.776,0.697,0.706,1.236,-0.522,-0.368,-0.261,-0.383,-0.059,-0.534,0.144,-0.258,-0.315,-0.163
98,1373158606,0.539,-0.050,-0.103,0.054,0.864,0.267,0.320,0.710,0.209,-1.311,1.678,-0.690,0.320,0.710,0.209,-1.311,1.678,-0.690,-0.009,-0.227,0.203,0.098,0.329,0.266,0.165,-0.297,0.071,0.351,0.330,-0.088,-0.019,0.210,0.264,0.312,0.124,0.381,0.414,0.396,0.176,0.041,0.008,-0.042,-0.404,-0.090,0.047,-0.291,-0.209,-0.233,0.000,0.000,-0.076,-0.073,0.093,0.093,-0.070,0.000,-0.067,0.049,-0.074,0.000,0.000,-0.072,0.094,-0.077,0.091,0.097,0.092,0.095,0.099,-0.048,-0.023,0.000,0.000,0.000,-0.052,0.000,0.000,0.000,-0.254,-0.359,-0.223,-0.227,0.317,0.348,-0.064,-0.408,-0.258,0.068,-0.084,-0.352,-0.364,-0.080,0.282,-0.162,0.524,0.199,0.355,0.238,0.188,-0.310,-0.044,-0.261,-0.383,-0.059,-0.334,0.144,-0.258,-0.315,0.088
31,1373158606,-0.269,-0.674,0.512,0.933,-0.670,-0.252,-0.294,-0.276,-1.702,6.674,-7.667,2.928,-0.294,-0.276,-1.702,6.674,-7.667,2.928,-0.009,-0.227,0.255,0.148,0.290,0.228,0.209,-0.297,0.115,0.271,0.379,-0.088,-0.019,0.257,0.227,0.364,0.009,0.348,0.380,0.364,0.174,0.061,-0.024,-0.042,-0.404,-0.090,0.074,-0.291,-0.209,-0.233,0.000,0.000,0.082,0.083,-0.102,-0.095,0.083,0.000,0.083,0.086,0.082,0.000,0.000,0.083,-0.099,0.082,-0.023,-0.113,-0.095,-0.101,-0.079,0.084,0.102,0.000,0.000,0.000,0.083,0.000,0.000,0.000,-0.254,-0.359,-0.286,-0.291,0.305,0.343,-0.129,-0.408,-0.320,-0.016,-0.147,-0.352,-0.364,-0.144,0.278,-0.224,0.393,0.184,0.337,0.222,0.224,-0.377,-0.160,-0.261,-0.383,-0.059,-0.399,0.144,-0.258,-0.315,-0.082
81,1373158606,-0.281,-0.918,0.501,-0.454,0.100,0.184,-0.284,-0.891,0.540,-0.986,1.108,-0.366,-0.284,-0.891,0.540,-0.986,1.108,-0.366,-0.009,-0.227,0.337,0.231,0.244,0.213,0.293,-0.297,0.199,0.357,0.461,-0.088,-0.019,0.340,0.219,0.446,0.008,0.283,0.338,0.299,0.033,0.146,0.085,-0.042,-0.404,-0.090,0.158,-0.291,-0.209,-0.233,0.000,0.000,0.071,0.071,-0.170,-0.166,0.072,0.000,0.071,0.079,0.072,0.000,0.000,0.072,-0.176,0.071,-0.023,-0.171,-0.084,-0.138,-0.039,0.071,0.076,0.000,0.000,0.000,0.070,0.000,0.000,0.000,-0.254,-0.359,-0.353,-0.347,0.365,0.457,-0.193,-0.408,-0.371,-0.081,-0.219,-0.352,-0.364,-0.211,0.401,-0.296,0.393,0.208,0.318,0.225,0.049,-0.411,-0.135,-0.261,-0.383,-0.059,-0.438,0.144,-0.258,-0.315,-0.069
41,1373158606,-0.873,-0.468,-0.674,-0.892,0.179,0.212,-0.840,-0.794,-0.054,0.031,-2.806,1.975,-0.840,-0.794,-0.054,0.031,-2.806,1.975,-0.009,-0.227,0.310,0.202,0.109,0.080,0.269,-0.297,0.169,0.351,0.438,-0.088,-0.019,0.316,0.079,0.420,0.008,0.914,0.144,0.810,0.034,0.116,0.062,-0.042,-0.404,-0.090,0.128,-0.291,-0.209,-0.233,0.000,0.000,0.178,0.

In [18]:
lambda_net_valid_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
69,1373158606,0.030,-0.349,-0.435,0.858,0.824,0.026,0.030,-0.365,-0.265,0.212,1.789,-0.458,0.030,-0.365,-0.265,0.212,1.789,-0.458,-0.009,-0.227,0.263,0.157,0.510,0.482,0.224,-0.297,0.124,0.646,0.391,-0.088,-0.019,0.270,0.500,0.373,0.394,0.575,0.587,0.581,0.503,0.070,0.004,-0.042,-0.404,-0.090,0.082,-0.291,-0.209,-0.233,0.000,0.000,-0.001,-0.001,-0.323,-0.337,-0.001,0.000,-0.001,-0.522,-0.001,0.000,0.000,-0.002,-0.379,-0.001,-0.353,-0.323,-0.200,-0.278,-0.432,-0.001,-0.023,0.000,0.000,0.000,-0.001,0.000,0.000,0.000,-0.254,-0.359,-0.270,-0.262,0.858,0.995,-0.108,-0.408,-0.284,0.740,-0.137,-0.352,-0.364,-0.127,0.982,-0.214,1.494,0.681,0.673,0.655,1.087,-0.314,-0.041,-0.261,-0.383,-0.059,-0.345,0.144,-0.258,-0.315,0.029
52,1373158606,-0.102,0.749,-0.652,0.583,-0.123,-0.650,-0.099,0.751,-0.960,1.984,-2.250,0.401,-0.099,0.751,-0.960,1.984,-2.250,0.401,-0.009,-0.227,0.388,0.341,0.315,0.252,0.401,-0.297,0.327,0.330,0.470,-0.088,-0.019,0.406,0.249,0.435,0.110,0.366,0.401,0.381,0.159,0.299,0.011,-0.042,-0.404,-0.090,0.304,-0.291,-0.209,-0.233,0.000,0.000,-0.209,-0.261,-0.000,-0.000,-0.325,0.000,-0.277,-0.001,-0.097,0.000,0.000,-0.255,-0.000,-0.167,-0.000,-0.001,-0.001,-0.001,-0.000,-0.210,-0.021,0.000,0.000,0.000,-0.266,0.000,0.000,0.000,-0.254,-0.359,-0.587,-0.814,0.293,0.321,-0.597,-0.408,-0.923,0.054,-0.273,-0.352,-0.364,-0.494,0.255,-0.383,0.479,0.177,0.334,0.216,0.153,-1.025,-0.046,-0.261,-0.383,-0.059,-1.101,0.144,-0.258,-0.315,-0.094
68,1373158606,-0.504,-0.747,-0.661,0.753,-0.040,-0.778,-0.495,-0.735,-1.845,6.343,-8.910,3.794,-0.495,-0.735,-1.845,6.343,-8.910,3.794,-0.009,-0.227,0.358,0.252,0.107,0.080,0.316,-0.297,0.219,0.389,0.483,-0.088,-0.019,0.362,0.079,0.467,0.008,0.849,0.251,0.582,0.034,0.166,0.113,-0.042,-0.404,-0.090,0.178,-0.291,-0.209,-0.233,0.000,0.000,0.110,0.110,-0.107,-0.081,0.114,0.000,0.110,0.127,0.112,0.000,0.000,0.113,-0.079,0.110,-0.023,-0.602,-0.000,-0.272,-0.039,0.109,0.121,0.000,0.000,0.000,0.109,0.000,0.000,0.000,-0.254,-0.359,-0.385,-0.383,0.135,0.188,-0.227,-0.408,-0.409,-0.114,-0.248,-0.352,-0.364,-0.243,0.122,-0.325,0.394,-0.960,0.163,-0.571,0.049,-0.452,-0.181,-0.261,-0.383,-0.059,-0.478,0.144,-0.258,-0.315,-0.105
43,1373158606,0.695,-0.680,0.394,-0.254,-0.310,0.513,0.655,-0.428,-0.083,0.188,-0.189,0.070,0.655,-0.428,-0.083,0.188,-0.189,0.070,-0.009,-0.227,0.335,0.236,0.211,0.148,0.302,-0.297,0.206,0.308,0.458,-0.088,-0.019,0.344,0.142,0.440,0.003,0.257,0.300,0.276,0.026,0.164,0.008,-0.042,-0.404,-0.090,0.171,-0.291,-0.209,-0.233,0.000,0.000,-0.003,-0.001,0.179,0.182,-0.002,0.000,-0.001,-0.002,-0.006,0.000,0.000,-0.003,0.187,-0.006,0.189,0.187,0.174,0.181,0.230,-0.001,-0.022,0.000,0.000,0.000,-0.001,0.000,0.000,0.000,-0.254,-0.359,-0.350,-0.361,0.299,0.353,-0.196,-0.408,-0.394,-0.038,-0.208,-0.352,-0.364,-0.209,0.291,-0.285,0.648,0.172,0.317,0.206,0.278,-0.462,-0.044,-0.261,-0.383,-0.059,-0.482,0.144,-0.258,-0.315,0.167
12,1373158606,0.414,0.021,-0.724,-0.319,-0.203,0.791,0.411,0.072,-0.767,-1.449,2.951,-1.372,0.411,0.072,-0.767,-1.449,2.951,-1.372,-0.009,-0.227,0.353,0.245,0.174,0.110,0.314,-0.297,0.212,0.415,0.492,-0.088,-0.019,0.368,0.105,0.472,-0.039,0.224,0.263,0.241,-0.015,0.163,0.015,-0.042,-0.404,-0.090,0.173,-0.291,-0.209,-0.233,0

In [19]:
lambda_net_test_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36,wb_37,wb_38,wb_39,wb_40,wb_41,wb_42,wb_43,wb_44,wb_45,wb_46,wb_47,wb_48,wb_49,wb_50,wb_51,wb_52,wb_53,wb_54,wb_55,wb_56,wb_57,wb_58,wb_59,wb_60,wb_61,wb_62,wb_63,wb_64,wb_65,wb_66,wb_67,wb_68,wb_69,wb_70,wb_71,wb_72,wb_73,wb_74,wb_75,wb_76,wb_77,wb_78,wb_79,wb_80,wb_81,wb_82,wb_83,wb_84,wb_85,wb_86,wb_87,wb_88,wb_89,wb_90
29,1373158606,-0.118,0.907,0.512,0.178,0.895,-0.481,-0.116,0.865,0.868,-0.966,2.406,-1.174,-0.116,0.865,0.868,-0.966,2.406,-1.174,-0.009,-0.227,0.134,0.085,0.592,0.489,0.114,-0.297,0.069,0.823,0.362,-0.088,-0.019,0.134,0.518,0.175,0.372,0.651,0.672,0.680,0.520,0.042,0.008,-0.042,-0.404,-0.090,0.048,-0.291,-0.209,-0.233,0.000,0.000,-0.135,-0.086,-0.002,-0.178,-0.114,0.000,-0.071,-0.512,-0.198,0.000,0.000,-0.133,-0.249,-0.175,-0.275,-0.143,-0.002,-0.022,-0.435,-0.048,-0.023,0.000,0.000,0.000,-0.054,0.000,0.000,0.000,-0.254,-0.359,-0.197,-0.227,0.558,0.622,-0.042,-0.408,-0.259,0.555,0.191,-0.352,-0.364,-0.031,0.625,-0.057,1.058,0.433,0.589,0.475,0.754,-0.310,-0.045,-0.261,-0.383,-0.059,-0.335,0.144,-0.258,-0.315,-0.112
38,1373158606,-0.309,-0.138,0.431,0.763,-0.642,-0.165,-0.305,-0.210,0.630,1.208,-2.415,1.111,-0.305,-0.210,0.630,1.208,-2.415,1.111,-0.009,-0.227,0.202,0.094,0.318,0.257,0.160,-0.297,0.061,0.339,0.329,-0.088,-0.019,0.207,0.251,0.312,0.121,0.374,0.418,0.395,0.170,0.007,-0.067,-0.042,-0.404,-0.090,0.020,-0.291,-0.209,-0.233,0.000,0.000,0.090,0.091,-0.076,-0.078,0.086,0.000,0.092,-0.007,0.086,0.000,0.000,0.086,-0.086,0.089,-0.046,-0.066,-0.024,-0.043,-0.065,0.094,0.110,0.000,0.000,0.000,0.094,0.000,0.000,0.000,-0.254,-0.359,-0.249,-0.271,0.312,0.347,-0.096,-0.408,-0.310,0.053,-0.102,-0.352,-0.364,-0.106,0.281,-0.181,0.531,0.192,0.350,0.231,0.192,-0.388,-0.171,-0.261,-0.383,-0.059,-0.405,0.144,-0.258,-0.315,-0.093
79,1373158606,0.076,-0.647,-0.778,0.686,0.008,-0.023,0.077,-0.700,-0.316,-0.777,1.909,-0.886,0.077,-0.700,-0.316,-0.777,1.909,-0.886,-0.009,-0.227,0.371,0.266,0.295,0.291,0.332,-0.297,0.233,0.416,0.498,-0.088,-0.019,0.378,0.298,0.481,0.008,0.220,0.248,0.232,0.034,0.173,0.134,-0.042,-0.404,-0.090,0.189,-0.291,-0.209,-0.233,0.000,0.000,-0.002,-0.001,-0.166,-0.205,-0.002,0.000,-0.001,-0.031,-0.010,0.000,0.000,-0.002,-0.239,-0.006,-0.024,-0.028,0.039,0.016,-0.041,-0.039,-0.001,0.000,0.000,0.000,-0.024,0.000,0.000,0.000,-0.254,-0.359,-0.377,-0.370,0.380,0.550,-0.216,-0.408,-0.392,-0.108,-0.244,-0.352,-0.364,-0.235,0.509,-0.321,0.394,0.019,0.184,0.061,0.049,-0.432,-0.154,-0.261,-0.383,-0.059,-0.458,0.144,-0.258,-0.315,0.063
19,1373158606,0.821,0.491,0.296,-0.117,-0.999,0.483,0.817,0.598,-0.504,2.233,-3.932,1.781,0.817,0.598,-0.504,2.233,-3.932,1.781,-0.009,-0.227,0.356,0.296,0.250,0.186,0.369,-0.297,0.067,0.220,0.494,-0.088,-0.019,0.390,0.179,0.442,0.044,0.293,0.340,0.313,0.067,0.040,0.008,-0.042,-0.404,-0.090,0.047,-0.291,-0.209,-0.233,0.000,0.000,-0.222,-0.227,0.213,0.212,-0.300,0.000,-0.069,0.237,-0.197,0.000,0.000,-0.272,0.219,-0.233,0.207,0.229,0.208,0.221,0.247,-0.047,-0.023,0.000,0.000,0.000,-0.052,0.000,0.000,0.000,-0.254,-0.359,-0.533,-0.651,0.344,0.390,-0.443,-0.408,-0.257,0.095,-0.317,-0.352,-0.364,-0.405,0.328,-0.407,0.628,0.220,0.365,0.255,0.279,-0.309,-0.044,-0.261,-0.383,-0.059,-0.334,0.144,-0.258,-0.315,0.186
27,1373158606,-0.122,0.643,0.922,0.104,-0.147,0.814,-0.116,0.426,2.753,-5.642,7.316,-2.572,-0.116,0.426,2.753,-5.642,7.316,-2.572,-0.009,-0.227,0.136,0.086,0.634,0.565,0.115,-0.297,0.070,0.897,0.424,-0.088,-0.019,0.135,0.593,0.175,0.455,0.719,0.731,0.746,0.635,0.042,0.008,-0.042,-0.404,-0.090,0.048,-0.291,-0.209,-0.233,0.000,0.000,-0

## Interpretation Network Training

In [ ]:
(history_list, 
scores_list, 

function_values_complete_list, 
function_values_valid_list, 
function_values_test_list, 

inet_preds_list, 
inet_preds_valid_list, 
inet_preds_test_list, 

distrib_dict_list,
model_list) = calculate_interpretation_net_results(lambda_net_train_dataset_list, 
                                                   lambda_net_valid_dataset_list, 
                                                   lambda_net_test_dataset_list)

Trial 2 Complete [00h 00m 13s]
val_loss: 0.47156694531440735

Best val_loss So Far: 0.18958021700382233
Total elapsed time: 00h 00m 29s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_block_1/ke...|3                 |3                 
conv_block_1/se...|False             |False             
conv_block_1/ma...|True              |True              
conv_block_1/dr...|0                 |0                 
conv_block_1/nu...|2                 |2                 
conv_block_1/nu...|2                 |2                 
conv_block_1/fi...|32                |32                
conv_block_1/fi...|512               |512               
conv_block_1/fi...|32                |32                
conv_block_1/fi...|32                |32                
dense_block_1/u...|False             |False             
dense_block_1/n...|2                 |2                 
dense_block_1/u...|32                |32                
dense_block_1/d...|0.25              |0

## Evaluate Interpretation Net

In [ ]:
if nas:
    for trial in history_list[-1]: 
        print(trial.summary())

In [ ]:
if len(model_list) >= 1:
    print(model_list[-1].summary())
    print(model_list[-1].get_config())

In [ ]:
scores_list[-1]

In [ ]:
distrib_dict_list[-1]['MAE']

In [ ]:
distrib_dict_list[-1]['R2']

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
polynomial_inet = inet_preds_test_list[-1][index_min]

print(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][index_min])

print_polynomial_from_coefficients(polynomial_inet)

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial

print(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][index_min])

polynomial_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index_min]
print_polynomial_from_coefficients(polynomial_lambda, force_complete_poly_representation=True)

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial
polynomial_target = lambda_net_test_dataset.target_polynomial_list[index_min]
print_polynomial_from_coefficients(polynomial_target, force_complete_poly_representation=True)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
polynomial_inet = inet_preds_test_list[-1][index_max]

print(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][index_max])

print_polynomial_from_coefficients(polynomial_inet)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial

print(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][index_max])

polynomial_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index_max]
print_polynomial_from_coefficients(polynomial_lambda, force_complete_poly_representation=True)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial
polynomial_target = lambda_net_test_dataset.target_polynomial_list[index_max]
print_polynomial_from_coefficients(polynomial_target, force_complete_poly_representation=True)

In [ ]:
r2_values_inet = distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test']
print('Mean: ' + str(np.mean(r2_values_inet)) + ' (' + str(r2_values_inet.shape[0]) + ' Samples)')

r2_values_positive_inet = r2_values_inet[r2_values_inet>0]
print('Mean (only positive): ' + str(np.mean(r2_values_positive_inet)) + ' (' + str(r2_values_positive_inet.shape[0]) + ' Samples)')



In [ ]:
r2_values_lstsq_lambda = distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test']
print('Mean: ' + str(np.mean(r2_values_lstsq_lambda)) + ' (' + str(r2_values_inet.shape[0]) + ' Samples)')

r2_values_positive_lstsq_lambda = r2_values_lstsq_lambda[r2_values_lstsq_lambda>0]
print('Mean (only positive): ' + str(np.mean(r2_values_positive_lstsq_lambda)) + ' (' + str(r2_values_positive_lstsq_lambda.shape[0]) + ' Samples)')



In [ ]:
sns.histplot(distrib_dict_list[-1]['MAE'].loc['inetPoly_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'] > -50])


In [ ]:
sns.histplot(distrib_dict_list[-1]['MAE'].loc['lstsqLambda_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'] > -50])


In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'] > -50], binwidth=0.2)
p.set(xlim=(-30, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'] > -50], binwidth=0.1)
p.set(xlim=(0, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'] > -50], binwidth=0.2)
p.set(xlim=(-10, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'] > -50], binwidth=0.1)
p.set(xlim=(0, 1))

In [ ]:
if not nas:
    history = history_list[-1]

    plt.plot(history[list(history.keys())[1]])
    if consider_labels_training or evaluate_with_real_function:
        plt.plot(history[list(history.keys())[len(history.keys())//2+1]]) 
    plt.title('model ' + list(history.keys())[1])
    plt.ylabel('metric')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig('./data/results/' + path_identifier_interpretation_net_data + '/metric_' + path_identifier_interpretation_net_data + '_epoch_' + str(epochs_lambda).zfill(3) + '.png')


In [ ]:
if not nas:
    history = history_list[-1]

    plt.plot(history['loss'])
    if consider_labels_training or evaluate_with_real_function:
        plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig('./data/results/' + path_identifier_interpretation_net_data + '/loss_' + path_identifier_interpretation_net_data + '_epoch_' + str(epochs_lambda).zfill(3) + '.png')    


### Multi Epoch/Sampes Analysis

### Generate Comparison Plots

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list)

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['MAE FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list)

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['R2 FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list, ylim=(-5, 1))

#### Generate and Analyze Predictions for Random Function

In [ ]:
index = 5

polynomial_target = lambda_net_test_dataset.target_polynomial_list[index]
polynomial_lstsq_target = lambda_net_test_dataset.lstsq_target_polynomial_list[index]
polynomial_lstsq_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index]
polynomial_inet = inet_preds_test_list[-1][index]

print('Target Poly:')
print_polynomial_from_coefficients(polynomial_target, force_complete_poly_representation=True, round_digits=4)
print('LSTSQ Target Poly:')
print_polynomial_from_coefficients(polynomial_lstsq_target, force_complete_poly_representation=True, round_digits=4)
print('LSTSQ Lambda Poly:')
print_polynomial_from_coefficients(polynomial_lstsq_lambda, force_complete_poly_representation=True, round_digits=4)
print('I-Net Poly:')
print_polynomial_from_coefficients(polynomial_inet, round_digits=4)


In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      inet_preds_test_list,
                                                      rand_index=index, 
                                                      plot_type=1)

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      inet_preds_test_list,
                                                      rand_index=index, 
                                                      plot_type=2)

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      inet_preds_test_list,
                                                      rand_index=index, 
                                                      plot_type=3)

# BENCHMARK (RANDOM GUESS) EVALUATION

In [ ]:
list_of_random_polynomials = np.random.uniform(low=-10, high=10, size=(len(lambda_net_test_dataset_list[-1]), sparsity))

In [ ]:
true_fv_test = parallel_fv_calculation_from_polynomial(lambda_net_test_dataset_list[-1].target_polynomial_list, lambda_net_test_dataset_list[-1].test_data_list, force_complete_poly_representation=True)
random_fv_test = parallel_fv_calculation_from_polynomial(list_of_random_polynomials, lambda_net_test_dataset_list[-1].test_data_list, force_complete_poly_representation=True)

In [ ]:
print('Random Guess Error Coefficients: ' + str(np.round(mean_absolute_error(lambda_net_test_dataset_list[-1].target_polynomial_list, list_of_random_polynomials), 4)))

In [ ]:
print('Random Guess Error FVs: ' + str(np.round(mean_absolute_error_function_values(true_fv_test, random_fv_test), 4)))

# BENCHMARK (EDUCATED GUESS/MEAN PREDICTION) EVALUATION

In [ ]:
true_fv_train = parallel_fv_calculation_from_polynomial(lambda_net_test_dataset_list[-1].target_polynomial_list, lambda_net_test_dataset_list[-1].test_data_list, force_complete_poly_representation=True)

mean_fv = np.mean(true_fv_train)
mean_fv_pred_test = [mean_fv for _ in range(true_fv_test.shape[0])]

In [ ]:
print('Educated Guess/Mean Prediction Error FVs: ' + str(np.round(mean_absolute_error_function_values(true_fv_test, mean_fv_pred_test), 4)))

In [ ]:
%%script false --no-raise-error

base_model = generate_base_model()
random_evaluation_dataset = np.random.uniform(low=x_min, high=x_max, size=(random_evaluation_dataset_size, n))
#random_evaluation_dataset = lambda_train_input_train_split[0]#lambda_train_input[0] #JUST [0] HERE BECAUSE EVALUATION ALWAYS ON THE SAME DATASET FOR ALL!!
list_of_monomial_identifiers_numbers = np.array([list(monomial_identifiers) for monomial_identifiers in list_of_monomial_identifiers]).astype(float)


loss_function = mean_absolute_error_tf_fv_lambda_extended_wrapper(random_evaluation_dataset, list_of_monomial_identifiers_numbers, base_model)      

X_train = X_train_list[-1].values[:,1:]
y_train = y_train_list[-1].values[:,2:]

#X_train = X_train[:,1:]
y_train_model = np.hstack((y_train, X_train))

print('seed_in_inet_training = ' + str(seed_in_inet_training), loss_function(y_train_model, y_train))


seed_in_inet_training = False

loss_function = mean_absolute_error_tf_fv_lambda_extended_wrapper(random_evaluation_dataset, list_of_monomial_identifiers_numbers, base_model)      

X_train = X_train_list[-1].values[:,1:]
y_train = y_train_list[-1].values[:,2:]

X_train = X_train[:,1:]
y_train_model = np.hstack((y_train, X_train))

print('seed_in_inet_training = ' + str(seed_in_inet_training), loss_function(y_train_model, y_train))

seed_in_inet_training = True



In [ ]:
if use_gpu:
    from numba import cuda 
    device = cuda.get_current_device()
    device.reset()